## Test model from hugging faces


In [37]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

In [38]:
# Load the pre-trained model and tokenizer from Hugging Face
model_name = "AleOfDurin/Sentiment_fine_tuned"

# Load the model and tokenizer from the Hugging Face Hub
model = AutoModelForSequenceClassification.from_pretrained(model_name)  # Ensure it's SequenceClassification model
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at AleOfDurin/Sentiment_fine_tuned and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:

#function to predict sentiment
def predict_sentiment(phrase):
    # Tokenize the input phrase and prepare it for the model
    inputs = tokenizer(phrase, padding=True, truncation=True, max_length=128, return_tensors="pt")

    # Put the model in evaluation mode
    model.eval()

    # Disable gradient calculations for inference
    with torch.no_grad():
        # Forward pass to get model outputs
        outputs = model(**inputs)
        logits = outputs.logits  # No need for CLS extraction here, AutoModelForSequenceClassification outputs logits directly
        
        # Compute the probabilities for each class (0 for negative, 1 for positive)
        probabilities = torch.softmax(logits, dim=-1)
    
    # Log the logits and probabilities
    print(f"Logits: {logits}")
    print(f"Probabilities: {probabilities}")

    # Extract the predicted class (0 for negative, 1 for positive)
    predicted_class = torch.argmax(probabilities).item()

    # Map the class to a label
    return "Positive" if predicted_class == 1 else "Negative"




In [44]:
# Example test with a phrase
input_phrase = "your song battlefield is addicting i really cant stop listening to it thank you for a great song,"  # Input phrase here
result = predict_sentiment(input_phrase)
print(f"Sentiment: {result}")

Logits shape: torch.Size([1, 2])
Logits: tensor([-0.0210, -0.2409])
Probabilities: tensor([0.5547, 0.4453])
Sentiment: Negative


In [45]:
# Define the function to predict sentiment for an input phrase
def predict_sentiment(phrase):
    # Tokenize the input phrase and prepare it for the model
    inputs = tokenizer(phrase, padding=True, truncation=True, max_length=128, return_tensors="pt")

    # Move inputs to the same device as the model (e.g., GPU or CPU)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Put the model in evaluation mode
    model.eval()

    # Disable gradient calculations for inference
    with torch.no_grad():
        # Forward pass to get model outputs
        outputs = model(**inputs)
        logits = outputs.logits
        
        # Check the shape of the logits to adapt the code
        print(f"Logits shape: {logits.shape}")

        if logits.ndim == 3:
            # If logits are 3D, we need to extract the CLS token logits [batch_size, sequence_length, num_labels]
            cls_logits = logits[0, 0, :]  # Extract the [CLS] token logits
        else:
            # If logits are 2D, we directly have [batch_size, num_labels]
            cls_logits = logits[0]

        # Compute the probabilities for each class (0 for negative, 1 for positive)
        probabilities = torch.softmax(cls_logits, dim=-1)

    # Log the logits and probabilities
    print(f"Logits: {cls_logits}")
    print(f"Probabilities: {probabilities}")

    # Extract the predicted class (0 for negative, 1 for positive)
    predicted_class = torch.argmax(probabilities).item()

    # Map the class to a label
    return "Positive" if predicted_class == 1 else "Negative"


# Example test with a phrase
input_phrase = "your song battlefield is addicting i really cant stop listening to it thank you for a great song,"
result = predict_sentiment(input_phrase)
print(f"Sentiment: {result}")


Logits shape: torch.Size([1, 2])
Logits: tensor([-0.0210, -0.2409])
Probabilities: tensor([0.5547, 0.4453])
Sentiment: Negative
